In [3]:
import init
from base import epsilon_rho, metrics
from base import optimize_algorithm as optimize
import numpy as np
import itertools
from scipy.linalg import eigh
import tensorflow as tf

In [4]:
def gellmann_matrices(d):
    """Generate generalized Gell-Mann matrices for SU(d)."""
    matrices = []
    
    # Off-diagonal symmetric and anti-symmetric
    for i in range(d):
        for j in range(i + 1, d):
            mat = np.zeros((d, d), dtype=complex)
            mat[i, j] = mat[j, i] = 1
            matrices.append(mat)

            mat = np.zeros((d, d), dtype=complex)
            mat[i, j] = -1j
            mat[j, i] = 1j
            matrices.append(mat)

    # Diagonal traceless matrices
    for k in range(1, d):
        diag_matrix = np.zeros((d, d), dtype=complex)
        for i in range(k):
            diag_matrix[i, i] = 1
        diag_matrix[k, k] = -k
        matrices.append(diag_matrix / np.sqrt(k * (k + 1)))

    return matrices

def diagonalize_gellmann(d):
    """Diagonalize the Gell-Mann matrices for SU(d)."""
    lambdas = gellmann_matrices(d)
    eigen_data = []

    for l in lambdas:
        eigvals, eigvecs = np.linalg.eigh(l) 
        eigen_data.append((eigvals, eigvecs)) 
    
    return eigen_data

diagonalize_gellmann(4)

[(array([-1.,  0.,  0.,  1.]),
  array([[-0.70710678+0.j,  0.        +0.j,  0.        +0.j,
           0.70710678+0.j],
         [ 0.70710678+0.j,  0.        +0.j,  0.        +0.j,
           0.70710678+0.j],
         [ 0.        +0.j,  1.        +0.j,  0.        +0.j,
           0.        +0.j],
         [ 0.        +0.j,  0.        +0.j,  1.        +0.j,
           0.        +0.j]])),
 (array([-1.,  0.,  0.,  1.]),
  array([[-0.70710678+0.j        ,  0.        +0.j        ,
           0.        +0.j        , -0.70710678+0.j        ],
         [ 0.        +0.70710678j,  0.        +0.j        ,
           0.        +0.j        ,  0.        -0.70710678j],
         [-0.        +0.j        ,  1.        +0.j        ,
           0.        +0.j        ,  0.        +0.j        ],
         [-0.        +0.j        ,  0.        +0.j        ,
           1.        +0.j        ,  0.        +0.j        ]])),
 (array([-1.,  0.,  0.,  1.]),
  array([[-0.70710678+0.j,  0.        +0.j,  0.        +0.j,


In [5]:
import random
def construct_measurement_operators():
    eigen_data = diagonalize_gellmann(2)
    projectors = []

    for _, eigenvectors in eigen_data:
        for i in range(eigenvectors.shape[1]):  # Each eigenstate
            vi = eigenvectors[:, i].reshape(-1, 1)  # |v_i⟩
            projector = vi @ vi.conj().T  # |v_i⟩⟨v_i|
            projectors.append(projector)
    # Ensure they sum to identity
    sum_projectors = sum(projectors)
    scale_factor = np.trace(sum_projectors) / np.trace(np.eye(2))  # Normalize to identity
    projectors = [M / scale_factor for M in projectors]

    return projectors

def generate_n_qubits_measurement_operators(n):
    single_qubit_M = construct_measurement_operators()
    M_n_qubit = []

    for combination in itertools.product(single_qubit_M, repeat=n):
        M = combination[0]
        for i in range(1, n):
            M = np.kron(M, combination[i])  # Extend to n-qubit
        M_n_qubit.append(M)
        

    return M_n_qubit

In [6]:
import random
def generate_n_qubits_rho(n):
    """Generate 6^n probe states for an n-qubit system."""
    d = 2**n 
    eigen_data = diagonalize_gellmann(d)
    
    # Extract eigenvalues and eigenvectors
    eigvals, eigvecs = zip(*eigen_data)
    eigvals = np.abs(eigvals)
    eigvals /= np.sum(eigvals)
    rho_list = []

    for _ in range(6**n):
        eig_set = random.choice(eigvals)  # Sample eigenvalues
        eig_diag = np.diag(eig_set) 

        eig_vec = eigvecs[0]
        rho = eig_vec @ eig_diag @ eig_vec.conj().T 
        rho /= np.trace(rho)  # Normalize

        rho_list.append(rho)

    return rho_list


In [7]:
n = 2

# Generate 6^n density matrices
rho_list = generate_n_qubits_rho(n)
print(f"Generated {len(rho_list)} of {rho_list[0].shape} rho.")

# Generate 6^n measurement operators
M_list = generate_n_qubits_measurement_operators(n)
print(f"Generated {len(M_list)} of {M_list[0].shape} M.")

# Generate epsilon
epsilon = init.create_unitary_matrix(num_qubits=n)
print(f"Generated {epsilon.shape} epsilon.")

# Generate K list
unitary = init.create_unitary_matrix(num_qubits=n)
kraus_operators = init.create_kraus_operators_from_unitary(unitary)
print(f"Generated {len(kraus_operators)} of {kraus_operators[0].shape} kraus operators.")

Generated 36 of (4, 4) rho.
Generated 36 of (4, 4) M.
Generated (4, 4) epsilon.
Generated 4 of (4, 4) kraus operators.


In [ ]:
def compute_simulated_data(M_list, rho_list, epsilon):
    """Compute d_{i,j} = Tr(M_j E_rand(rho_i))"""

    data = np.zeros((len(rho_list), len(M_list)), dtype=complex)
    
    for i, rho in enumerate(rho_list):
        rho_transformed = epsilon_rho.calculate_from_unitary(rho=rho, unitary_matrix=epsilon)  # Apply channel
        for j, M in enumerate(M_list):
            data[i, j] = np.trace(M @ rho_transformed)  # Compute expectation value
    
    return data

# Compute and print the simulated data
simulated_data = compute_simulated_data(M_list, rho_list, epsilon)
print("(d_{i,j}):\n", simulated_data.shape)

(d_{i,j}):
 (36, 36)


In [338]:
def loss_function(d_ij, M_list, rho_list, kraus_operators):
    """Compute loss = Σ (d_{i,j} - Tr(M_j * E(rho_i)))^2"""
    
    loss = tf.constant(0.0, dtype=tf.complex128)  # Initialize loss
    
    for i, rho in enumerate(rho_list):
        rho_transformed = epsilon_rho.calculate_from_kraus_operators(
            rho=rho, kraus_operators=kraus_operators
        )  # Apply channel
        
        for j, M in enumerate(M_list): 
            predicted = tf.linalg.trace(tf.linalg.matmul(M, rho_transformed))  # Tr( M_j * rho_i' )
            diff = d_ij[i, j] - predicted
            loss += diff ** 2  # Squared absolute error
    
    return tf.math.real(loss)
loss_value = loss_function(simulated_data, M_list, rho_list, kraus_operators)

print("Loss function value:", loss_value)

Loss function value: tf.Tensor(0.17783323225306555, shape=(), dtype=float64)


In [ ]:

def calculate_adam_kraus(M_list, rho_list, unitary, kraus_operators, m, v, num_qubits, t, alpha=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8, mode = 'fidelity'):
    tensorKraus = tf.Variable(kraus_operators, dtype=tf.complex128)

    beta1 = tf.constant(beta1, dtype=tf.complex128)
    beta2 = tf.constant(beta2, dtype=tf.complex128)
    t = tf.constant(t, dtype=tf.complex128)

    with tf.GradientTape() as tape:
        data = compute_simulated_data(M_list, rho_list, unitary)
        f = loss_function(data, M_list, rho_list, tensorKraus)
    
    # Calculate the gradient
    c = tape.gradient(f, tensorKraus)
    
    # Calculate projection
    proj = c - tensorKraus @ (np.transpose(np.conjugate(c)) @ tensorKraus + np.transpose(np.conjugate(tensorKraus)) @ c) / 2

    # Update Adam variables
    m = beta1 * m + (1 - beta1) * proj
    v = beta2 * v + (1 - beta2) * tf.math.square(proj)

    # Bias correction
    m_hat = m / (1 - tf.pow(beta1, t + 1))
    v_hat = v / (1 - tf.pow(beta2, t + 1))

    # Update the Kraus operators using Adam update rule
    updated_kraus_operators = tensorKraus - alpha * m_hat / (tf.math.sqrt(v_hat) + epsilon)
    return updated_kraus_operators, m, v, f

In [343]:
def optimize_adam_kraus(M_list, rho_list, unitary, kraus_operators, num_qubits, alpha=0.001, beta1 = 0.9, beta2 = 0.999, epsilon = 1e-8, num_loop=1000):
    kraus_operators_copy = tf.identity(kraus_operators)
    
    # Initialize m, v to zero matrices
    m = tf.zeros_like(kraus_operators_copy, dtype=tf.complex128)
    v = tf.zeros_like(kraus_operators_copy, dtype=tf.complex128)
    
    # Initialize a dictionary to track cost at each iteration
    cost_dict = []

    # Try looping manually
    for i in range(num_loop):
        kraus_operators_copy, m, v, cost = calculate_adam_kraus(M_list, rho_list, unitary, kraus_operators_copy, m = m, v = v, num_qubits=num_qubits, t = i, alpha=alpha, beta1=beta1, beta2=beta2, epsilon=epsilon)
        print(cost)
        
        
        # Store the cost for this iteration
        cost_dict.append(cost.numpy().real)
        
        # Reshape the matrices
        kraus_operators_copy = renormalize_kraus(kraus_operators_copy)
        check_kraus_validity(kraus_operators_copy)
        m = tf.reshape(m, (2**num_qubits, 2**num_qubits, 2**num_qubits))
        v = tf.reshape(v, (2**num_qubits, 2**num_qubits, 2**num_qubits))

    return kraus_operators_copy, cost_dict

In [344]:
kraus_operators_res, cost_dict = optimize_adam_kraus(M_list, rho_list, epsilon, kraus_operators, n, 0.05, num_loop=200)
print(kraus_operators_res)
print(cost_dict)

tf.Tensor(0.17783323225306555, shape=(), dtype=float64)
tf.Tensor(0.09305786270296945, shape=(), dtype=float64)
tf.Tensor(0.050310479091792634, shape=(), dtype=float64)
tf.Tensor(0.032338175607970567, shape=(), dtype=float64)
tf.Tensor(0.024975155813595898, shape=(), dtype=float64)
tf.Tensor(0.020208018707909663, shape=(), dtype=float64)
tf.Tensor(0.015199864816154285, shape=(), dtype=float64)
tf.Tensor(0.010384734742317174, shape=(), dtype=float64)
tf.Tensor(0.006995008541114377, shape=(), dtype=float64)
tf.Tensor(0.005480342717166463, shape=(), dtype=float64)
tf.Tensor(0.005087266526334199, shape=(), dtype=float64)
tf.Tensor(0.0048865623346148876, shape=(), dtype=float64)
tf.Tensor(0.004579572056489723, shape=(), dtype=float64)
tf.Tensor(0.004245657580634725, shape=(), dtype=float64)
tf.Tensor(0.003947091609232722, shape=(), dtype=float64)
tf.Tensor(0.003621485302697546, shape=(), dtype=float64)
tf.Tensor(0.0032214604717145734, shape=(), dtype=float64)
tf.Tensor(0.0028137271043520745

In [ ]:
def project_density_matrix(rho):
    """Ensure rho is Hermitian and positive semi-definite."""
    rho = (rho + tf.linalg.adjoint(rho)) / 2  # Make it Hermitian
    eigvals, eigvecs = tf.linalg.eigh(rho)  # Eigen decomposition
    
    eigvals = tf.cast(tf.abs(eigvals) + 1e-10, dtype=tf.complex128)  # Convert to complex128
    
    return eigvecs @ tf.linalg.diag(eigvals) @ tf.linalg.adjoint(eigvecs)


In [ ]:
def compute_simulated_data_our_method(kraus_operators, rho_list, epsilon):
    """Compute rho_f_i = E_rand(sum(K@rho_i@K_dagger))"""

    data = []
    
    for i, rho in enumerate(rho_list):

        rho2 = epsilon_rho.calculate_from_kraus_operators(rho, kraus_operators)
        data.append(epsilon_rho.calculate_from_unitary_dagger(rho2, epsilon))
        
    return data 

# Compute and print the simulated data
simulated_data_our_method = compute_simulated_data_our_method(kraus_operators, rho_list, epsilon)
rho_list_projected = []
for rho in rho_list:
    rho_list_projected.append(project_density_matrix(rho))
print("(rho_f_i):\n", simulated_data_our_method[0])

(rho_f_i):
 tf.Tensor(
[[ 0.25699585+0.j  0.00137585+0.j -0.00268149+0.j  0.00093819+0.j]
 [ 0.00137585+0.j  0.25186763+0.j  0.00401742+0.j  0.0105701 +0.j]
 [-0.00268149+0.j  0.00401742+0.j  0.25728767+0.j -0.01557206+0.j]
 [ 0.00093819+0.j  0.0105701 +0.j -0.01557206+0.j  0.23384885+0.j]], shape=(4, 4), dtype=complex128)


In [386]:

def loss_function_our_method(rho_f_list, rho_list):
    """Compute loss = 1 - mean(Fidelity)"""

    fidelity_sum = tf.constant(0.0, dtype=tf.complex128)

    for rho, rho_f in zip(rho_list, rho_f_list):
        rho_f = project_density_matrix(rho_f)

        sqrt_rho = tf.linalg.sqrtm(rho)
        intermediate = sqrt_rho @ rho_f @ sqrt_rho

        fidelity = tf.linalg.trace(tf.linalg.sqrtm(intermediate)) ** 2
        fidelity_sum += fidelity

    fidelity_avg = fidelity_sum / len(rho_list)

    return 1 - fidelity_avg  # Loss should approach 0 for perfect matching

loss_value = loss_function_our_method(simulated_data,  rho_list_projected)

print("Loss function value:", loss_value)

Loss function value: tf.Tensor((0.41341797512774225+0j), shape=(), dtype=complex128)


In [358]:
def renormalize_kraus(kraus_operators):
    """Ensure Kraus operators satisfy Σ K_i^† K_i = I"""
    summation = sum(tf.linalg.adjoint(K) @ K for K in kraus_operators)
    sqrt_inv = tf.linalg.inv(tf.linalg.sqrtm(summation))  # (Σ K_i† K_i)^(-1/2)
    return [K @ sqrt_inv for K in kraus_operators]


In [ ]:
import os

import warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Hide TensorFlow warnings
tf.get_logger().setLevel('ERROR')  # Hide logs
warnings.filterwarnings("ignore", category=UserWarning)
def calculate_adam_kraus_our_method(rho_list, unitary, kraus_operators, m, v, num_qubits, t, alpha=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8, mode = 'fidelity'):
    tensorKraus = tf.Variable(kraus_operators, dtype=tf.complex128)
    
    beta1 = tf.constant(beta1, dtype=tf.complex128)
    beta2 = tf.constant(beta2, dtype=tf.complex128)
    t = tf.constant(t, dtype=tf.complex128)

    with tf.GradientTape() as tape:
        data = compute_simulated_data_our_method(tensorKraus, rho_list, unitary)
        f = loss_function_our_method(data, rho_list)

    
    # Calculate the gradient
    c = tape.gradient(f, tensorKraus)
    

    # Calculate projection
    proj = c - tensorKraus @ (np.transpose(np.conjugate(c)) @ tensorKraus + np.transpose(np.conjugate(tensorKraus)) @ c) / 2

    # Update Adam variables
    m = beta1 * m + (1 - beta1) * proj
    v = beta2 * v + (1 - beta2) * tf.math.square(proj)

    # Bias correction
    m_hat = m / (1 - tf.pow(beta1, t + 1))
    v_hat = v / (1 - tf.pow(beta2, t + 1))

    # Update the Kraus operators using Adam update rule
    updated_kraus_operators = tensorKraus - alpha * m_hat / (tf.math.sqrt(v_hat) + epsilon)
    return updated_kraus_operators, m, v, f

In [378]:
def optimize_adam_kraus_our_method(rho_list, unitary, kraus_operators, num_qubits, alpha=0.001, beta1 = 0.9, beta2 = 0.999, epsilon = 1e-8, num_loop=1000):
    kraus_operators_copy = tf.identity(kraus_operators)
    
    # Initialize m, v to zero matrices
    m = tf.zeros_like(kraus_operators_copy, dtype=tf.complex128)
    v = tf.zeros_like(kraus_operators_copy, dtype=tf.complex128)
    
    # Initialize a dictionary to track cost at each iteration
    cost_dict = []

    # Try looping manually
    for i in range(num_loop):

        kraus_operators_copy, m, v, cost = calculate_adam_kraus_our_method(rho_list, unitary, kraus_operators_copy, m = m, v = v, num_qubits=num_qubits, t = i, alpha=alpha, beta1=beta1, beta2=beta2, epsilon=epsilon)
        print(cost)
        
        kraus_operators_copy = renormalize_kraus(kraus_operators_copy)
        # Store the cost for this iteration
        cost_dict.append(cost.numpy().real)
        check_kraus_validity(kraus_operators_copy)
        # Reshape the matrices
        m = tf.reshape(m, (2**num_qubits, 2**num_qubits, 2**num_qubits))
        v = tf.reshape(v, (2**num_qubits, 2**num_qubits, 2**num_qubits))

    return kraus_operators_copy, cost_dict

In [390]:
kraus_operators_res_our_method, cost_dict_our_method = optimize_adam_kraus_our_method(rho_list_projected, epsilon, kraus_operators, n, alpha=0.1, num_loop=200)
print(kraus_operators_res_our_method)
print(cost_dict_our_method)

tf.Tensor((0.4134179748917519+0j), shape=(), dtype=complex128)
tf.Tensor((0.18100248163697708+0j), shape=(), dtype=complex128)
tf.Tensor((0.060828356411600226+0j), shape=(), dtype=complex128)
tf.Tensor((0.037458202141911645+0j), shape=(), dtype=complex128)
tf.Tensor((0.04374377068674429+0j), shape=(), dtype=complex128)
tf.Tensor((0.04175795323982667+0j), shape=(), dtype=complex128)
tf.Tensor((0.03358849827923982+0j), shape=(), dtype=complex128)
tf.Tensor((0.025819915855206044+0j), shape=(), dtype=complex128)
tf.Tensor((0.019471025049422663+0j), shape=(), dtype=complex128)
tf.Tensor((0.013644671612046855+0j), shape=(), dtype=complex128)
tf.Tensor((0.009060756248809643+0j), shape=(), dtype=complex128)
tf.Tensor((0.006806382216344331+0j), shape=(), dtype=complex128)
tf.Tensor((0.0064593107861783405+0j), shape=(), dtype=complex128)
tf.Tensor((0.006267666612341127+0j), shape=(), dtype=complex128)
tf.Tensor((0.005193632196932696+0j), shape=(), dtype=complex128)
tf.Tensor((0.00372661699781162

In [515]:
rho_test = init.create_random_state(2)

rho2 = epsilon_rho.calculate_from_unitary(rho_test, epsilon)

rho2_ours = epsilon_rho.calculate_from_kraus_operators(rho_test, kraus_operators_res_our_method)

rho2_theirs = epsilon_rho.calculate_from_kraus_operators(rho_test, kraus_operators_res)

print (rho2)

print ("Their method: \n")
print (rho2_theirs)
print(metrics.compilation_trace_fidelity(rho2, rho2_theirs))

print ("Our method: \n")
print (rho2_ours)
print(metrics.compilation_trace_fidelity(rho2, rho2_ours))

[[ 0.36428774+0.j  0.22670344+0.j  0.26881254+0.j -0.32852329+0.j]
 [ 0.22670344+0.j  0.14108201+0.j  0.16728734+0.j -0.20444652+0.j]
 [ 0.26881254+0.j  0.16728734+0.j  0.19836018+0.j -0.2424215 +0.j]
 [-0.32852329+0.j -0.20444652+0.j -0.2424215 +0.j  0.29627006+0.j]]
Their method: 

tf.Tensor(
[[ 0.42156235-1.39629921e-33j  0.26625796+1.64753713e-17j
   0.24474124+5.17683314e-17j -0.30055094-3.57646213e-17j]
 [ 0.26625796-1.64753713e-17j  0.1932767 -6.98149605e-33j
   0.15311784+2.25534334e-17j -0.16451771-1.02991671e-17j]
 [ 0.24474124-5.17683314e-17j  0.15311784-2.25534334e-17j
   0.14307857+6.16297582e-33j -0.17703005+1.56894502e-17j]
 [-0.30055094+3.57646213e-17j -0.16451771+1.02991671e-17j
  -0.17703005-1.56894502e-17j  0.24208237-1.08333559e-32j]], shape=(4, 4), dtype=complex128)
tf.Tensor((0.9350501756822047-1.0339757656912846e-25j), shape=(), dtype=complex128)
Our method: 

tf.Tensor(
[[ 0.37308374+0.j  0.27461381+0.j  0.22802612+0.j -0.30679639+0.j]
 [ 0.27461381+0.j  0.21257

In [242]:
def check_rho_properties(rho_list):
    for i, rho in enumerate(rho_list):
        is_hermitian = np.allclose(rho, np.transpose(np.conjugate(rho)))  # Check if ρ = ρ†
        eigenvalues = np.linalg.eigvalsh(rho)  # Compute eigenvalues
        is_positive_semidefinite = np.all(eigenvalues >= -1e-10)  # Allow small numerical errors
        trace_one = np.isclose(np.trace(rho), 1)  # Check if Tr(ρ) ≈ 1
        
        print(f"ρ[{i}]: Hermitian = {is_hermitian}, Positive Semi-definite = {is_positive_semidefinite}, Trace = {np.trace(rho)}")
        print(f"Eigenvalues: {eigenvalues}\n")

check_rho_properties(rho_list)


ρ[0]: Hermitian = True, Positive Semi-definite = True, Trace = (1+0j)
Eigenvalues: [0.  0.  0.5 0.5]

ρ[1]: Hermitian = True, Positive Semi-definite = True, Trace = (1+0j)
Eigenvalues: [0.  0.  0.5 0.5]

ρ[2]: Hermitian = True, Positive Semi-definite = True, Trace = (1+0j)
Eigenvalues: [0.  0.  0.5 0.5]

ρ[3]: Hermitian = True, Positive Semi-definite = True, Trace = (1+0j)
Eigenvalues: [0.  0.  0.5 0.5]

ρ[4]: Hermitian = True, Positive Semi-definite = True, Trace = (1+0j)
Eigenvalues: [0.  0.  0.5 0.5]

ρ[5]: Hermitian = True, Positive Semi-definite = True, Trace = (1+0j)
Eigenvalues: [0.  0.  0.5 0.5]

ρ[6]: Hermitian = True, Positive Semi-definite = True, Trace = (1+0j)
Eigenvalues: [0.  0.  0.5 0.5]

ρ[7]: Hermitian = True, Positive Semi-definite = True, Trace = (1+0j)
Eigenvalues: [0.  0.  0.5 0.5]

ρ[8]: Hermitian = True, Positive Semi-definite = True, Trace = (1+0j)
Eigenvalues: [0.   0.25 0.25 0.5 ]

ρ[9]: Hermitian = True, Positive Semi-definite = True, Trace = (1+0j)
Eigenval

In [323]:
import tensorflow as tf
import numpy as np

def check_kraus_validity(kraus_operators):
    """Check if sum(K_i^† K_i) = I"""
    dim = kraus_operators[0].shape[0]  # Kích thước của ma trận
    identity = tf.eye(dim, dtype=tf.complex128)  # Ma trận đơn vị
    summation = sum(tf.linalg.adjoint(K) @ K for K in kraus_operators)  # ∑ K_i^† K_i

    # So sánh với ma trận đơn vị
    error = tf.linalg.norm(summation - identity).numpy().real
    
    if np.isclose(error, 0, atol=1e-6):
        return
    else:
        print("❌ Invalid Kraus operators! The sum does not equal identity.")

# Kiểm tra
check_kraus_validity(kraus_operators)


In [ ]:
import numpy as np

def check_valid_measurements(M_list, d):
    """
    Kiểm tra xem danh sách các measurement operators M_list có hợp lệ không.
    
    Điều kiện hợp lệ:
    1. M_i phải Hermitian
    2. M_i phải Positive Semi-Definite (PSD)
    3. Tổng M_i phải bằng ma trận đơn vị I (trong trường hợp POVM)
    
    Args:
        M_list: Danh sách các phép đo
        d: Kích thước không gian Hilbert (ví dụ: d=2^n cho hệ n qubit)
        
    Returns:
        None (In kết quả)
    """
    I = np.eye(d)  # Ma trận đơn vị cùng kích thước

    completeness_check = np.zeros((d, d), dtype=complex)  # Tổng của các M_i

    for i, M in enumerate(M_list):
        is_hermitian = np.allclose(M, M.conj().T)  # Kiểm tra Hermitian
        eigvals = np.linalg.eigvalsh(M)  # Lấy eigenvalues
        is_psd = np.all(eigvals >= -1e-10)  # Kiểm tra PSD (chấp nhận sai số nhỏ)

        completeness_check += M  # Cộng vào tổng

        print(f"M[{i}]: Hermitian = {is_hermitian}, Positive Semi-Definite = {is_psd}")
        print(f"  Eigenvalues: {eigvals}\n")

    # Kiểm tra tổng M_i có bằng I không (chỉ cần thiết nếu là POVM)
    is_complete = np.allclose(completeness_check, I)
    print(completeness_check)
    print(f"Completeness Check (ΣM_i = I): {is_complete}")
check_valid_measurements(M_list, 2**n)
